## Mount Drive & Unzip Dataset  
Load Google Drive and extract the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/LOOCV_2models_VIT/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT.zip /content
!unzip /content/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT.zip -d /content > /dev/null

## Select & Hold Out One Gîte  
Randomly choose one site and move its images to validation.

## And

## Define Paths & Seed  
Set up directory paths and a fixed random seed.

Here’s the expected directory layout for your dataset:

- **train/**: contains two class subfolders  
  - **background/** – all “negative” images  
  - **bats/**       – all “positive” images  
- **val/**: same structure, holds your held-out gîte images  
- **test/**: same structure, for final evaluation  


In [ ]:
import os
import random
import shutil

# Base path
base_path = "/content/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT"
train_dir = os.path.join(base_path, "train")
val_dir = os.path.join(base_path, "val")

background_dir = os.path.join(train_dir, "background")
bats_dir = os.path.join(train_dir, "bats")

val_background_dir = os.path.join(val_dir, "background")
val_bats_dir = os.path.join(val_dir, "bats")

# Fixed seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Gîte names
chosen_gites = [
    'Pont_de_Bousval_Photos_2022_PHOTO',
    'Modave_Camera_3_toiture_PHOTO',
    'Bornival_PHOTO_2023CAM04',
    'Pont_de_Bousval_Photos_2023_PHOTO_WK6HDBOUSVAL',
    'Pont_de_Bousval_Photos_2023_PHOTO_2022CAM12',
    'Pont_de_Bousval_Photos_2023_PHOTO_2023CAM06',
    'Bornival_PHOTO_2023CAM03',
    'Chaumont_Gistoux_Camera_2',
    'Chaumont_Gistoux_Camera_1',
    'Pont_de_Bousval_Photos_2023_PHOTO_2023CAM05',
    #'Anthisnes_Chateau_de_Xhos_Camera_1_HIT',
    'Jenneret_Camera_1_PHOTO',
    'Modave_Camera_plancher_PHOTO'
]

# Randomly select a gîte using fixed seed
held_out_gite = random.choice(chosen_gites)
print(f"📦 Holding out gîte for validation: {held_out_gite}")
print(f"🧪 Reproducible with seed: {RANDOM_SEED}")

# Create val folders
os.makedirs(val_background_dir, exist_ok=True)
os.makedirs(val_bats_dir, exist_ok=True)

# Function to move matching files
def move_files_by_gite(source_dir, target_dir, gite_name):
    moved_count = 0
    for fname in sorted(os.listdir(source_dir)):  # sort to ensure order
        if gite_name in fname:
            shutil.move(os.path.join(source_dir, fname), os.path.join(target_dir, fname))
            moved_count += 1
    return moved_count

# Move files
bkg_moved = move_files_by_gite(background_dir, val_background_dir, held_out_gite)
bats_moved = move_files_by_gite(bats_dir, val_bats_dir, held_out_gite)

print(f"✅ Moved {bkg_moved} background images and {bats_moved} bat images to validation set.")


📦 Holding out gîte for validation: Jenneret_Camera_1_PHOTO
🧪 Reproducible with seed: 42
✅ Moved 6889 background images and 1330 bat images to validation set.


## Imports & Configuration  
Import libraries and define training parameters.

In [ ]:
import math, json, numpy as np, torch
from PIL import Image
from collections import Counter
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import InterpolationMode
from timm.data.mixup import Mixup

# Paths & hyperparameters
DATA_DIR      = base_path
TRAIN_DIR     = os.path.join(DATA_DIR, "train")
VAL_DIR       = os.path.join(DATA_DIR, "val")
TEST_DIR      = os.path.join(DATA_DIR, "test")
INFER_DIR     = TEST_DIR
BATCH_SIZE    = 32
NUM_EPOCHS    = 1
LEARNING_RATE = 3e-5
OUTPUT_DIR    = "/content/efficientnet_finetuned_bats"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

# Mixed precision setup
from torch.cuda.amp import autocast, GradScaler
AMP_KW = dict(device_type="cuda")
scaler = GradScaler()

# Seed everything
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True

## Data Augmentation & Mixup  
Specify train/validation transforms and MixUp augmentation.

In [ ]:
weights   = EfficientNet_B0_Weights.IMAGENET1K_V1
normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5,1.0), ratio=(0.9,1.1), interpolation=InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.RandomPerspective(0.3, p=0.3),
    transforms.GaussianBlur(3, sigma=(0.1,2.0)),
    transforms.ToTensor(),
    normalize,
    transforms.RandomErasing(scale=(0.02,0.2), ratio=(0.3,3.3), p=0.25),
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize,
])

mixup_fn = Mixup(
    num_classes=2, mixup_alpha=0.2, cutmix_alpha=1.0,
    prob=1.0, switch_prob=0.5, mode="batch", label_smoothing=0.0
)


## Focal Loss Definition  
Implement a drop-in Focal Loss module.


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = torch.tensor(alpha) if isinstance(alpha, (list, tuple)) else alpha
        self.gamma, self.reduction = gamma, reduction

    def forward(self, logits, targets):
        ce = F.cross_entropy(logits, targets, reduction="none")
        pt = torch.exp(-ce)
        a = self.alpha.to(logits.device)[targets] if isinstance(self.alpha, torch.Tensor) else self.alpha
        loss = a * (1 - pt) ** self.gamma * ce
        return loss.mean() if self.reduction=="mean" else loss.sum()


## Model Setup  
Load EfficientNet-B0, freeze early layers, and replace the classifier.


In [ ]:
base_model = efficientnet_b0(weights=weights)
freeze_upto = len(base_model.features) * 2 // 3
for i, block in enumerate(base_model.features):
    if i < freeze_upto:
        for p in block.parameters():
            p.requires_grad = False

base_model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(base_model.classifier[1].in_features, 2)
)

try:
    model = torch.compile(base_model.to(DEVICE))
except:
    model = base_model.to(DEVICE)


## Optimizer & Scheduler  
Configure loss, optimizer, and learning-rate schedule.


In [ ]:
alpha     = torch.tensor([0.05, 0.95])
loss_fn   = FocalLoss(alpha=alpha, gamma=2.0).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS * len(train_loader))


## Training & Validation Loop  
Train for epochs, track metrics, and save the best model.


In [ ]:
best_f1, patience, epochs_no_imp = 0.0, 3, 0

for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    total_train = 0
    for x, y in tqdm(train_loader, desc=f"Train {epoch}/{NUM_EPOCHS}"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        x, y = mixup_fn(x, y)
        with autocast(**AMP_KW):
            logits = model(x)
            loss   = loss_fn(logits, y)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        total_train += loss.item()
    avg_train = total_train / len(train_loader)

    model.eval()
    val_losses, all_logits, all_labels = 0.0, [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Valid", leave=False):
            x, y = x.to(DEVICE), y.to(DEVICE)
            with autocast(**AMP_KW):
                out  = model(x)
                loss = loss_fn(out, y)
            val_losses += loss.item()
            all_logits.append(out.cpu())
            all_labels.append(y.cpu())
    avg_val = val_losses / len(val_loader)

    logits  = torch.cat(all_logits)
    labels  = torch.cat(all_labels)
    preds   = logits.argmax(1)
    f1_m    = f1_score(labels, preds, average="macro")

    print(f"Epoch {epoch}: TrainL={avg_train:.4f} ValL={avg_val:.4f} F1={f1_m:.4f}")

    if f1_m > best_f1:
        best_f1, epochs_no_imp = f1_m, 0
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_model.pth"))
        torch.save({"logits": logits, "labels": labels},   os.path.join(OUTPUT_DIR, "val_blob.pt"))
    else:
        epochs_no_imp += 1
        if epochs_no_imp >= patience:
            print("Early stopping.")
            break


## Threshold Tuning & ROC  
Find the optimal decision threshold and plot the ROC curve.


In [ ]:
blob   = torch.load(os.path.join(OUTPUT_DIR, "val_blob.pt"))
labels = blob["labels"].numpy()
probs  = blob["logits"].softmax(1)[:,1].numpy()

prec, rec, thr = precision_recall_curve(labels, probs)
best_thr = thr[np.argmin(np.abs(prec-rec))]
with open(os.path.join(OUTPUT_DIR, "best_thr.txt"), "w") as f:
    f.write(f"{best_thr:.4f}")

fpr, tpr, roc_thr = roc_curve(labels, probs)
roc_auc = auc(fpr, tpr)
idx     = np.argmin(np.abs(roc_thr-best_thr))

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.scatter(fpr[idx], tpr[idx], s=60, label=f"Thr={best_thr:.3f}")
plt.plot([0,1],[0,1],"k--",alpha=0.4)
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.title("ROC Curve"); plt.legend(loc="lower right")
plt.grid(True); plt.tight_layout(); plt.show()


## Inference on Test Crops  
Run the trained model on test images.


In [ ]:
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "best_model.pth")))
model.eval()
best_thr = float(open(os.path.join(OUTPUT_DIR, "best_thr.txt")).read())

for fn in os.listdir(INFER_DIR):
    if not fn.lower().endswith((".png",".jpg",".jpeg",".bmp",".tiff")):
        continue
    img    = Image.open(os.path.join(INFER_DIR, fn)).convert("RGB")
    tensor = val_transform(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad(), autocast(**AMP_KW):
        prob = model(tensor).softmax(1)[0,1].item()
    pred = "bats" if prob >= best_thr else "background"
    print(f"{fn}: {pred} (Pbat={prob:.3f})")



## Final Test Evaluation  
Evaluate on the independent test set and report metrics.

In [ ]:
test_set    = ImageFolder(TEST_DIR, transform=val_transform)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE*2, shuffle=False,
                         num_workers=num_workers, pin_memory=True,
                         persistent_workers=True)

all_logits, all_labels = [], []
with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Test", leave=False):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out  = model(x)
        all_logits.append(out.cpu())
        all_labels.append(y.cpu())

logits = torch.cat(all_logits); labels = torch.cat(all_labels)
probs  = logits.softmax(1)[:,1]
preds  = (probs >= best_thr).int()

precision = precision_score(labels, prefs, average="macro")
recall    = recall_score(labels, preds, average="macro")
f1_score  = f1_score(labels, preds, average="macro")
fpr, tpr, _ = roc_curve(labels, probs)
roc_auc    = auc(fpr, tpr)

print(f"Test precision={precision:.4f} recall={recall:.4f} F1={f1_score:.4f} AUC={roc_auc:.4f}")
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
plt.plot([0,1],[0,1],'k--',alpha=0.3)
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.title("ROC - Test Set"); plt.legend(loc="lower right")
plt.grid(True); plt.tight_layout(); plt.show()


# EfficientNet-B0 Bat Classifier (Google Colab Setup)

This script trains and evaluates a binary image classifier to distinguish bats from background images using EfficientNet-B0.

## Features
- Mixed-precision training using `torch.amp.autocast` for speed and efficiency
- `torch.compile()` for accelerated model execution (when supported)
- Balanced sampling and MixUp/CutMix augmentations for robust training
- Drop-in Focal Loss implementation to handle class imbalance
- Automatic threshold tuning with ROC/PR curve analysis
- Final performance metrics and visualization (AUC, F1, Precision, Recall)

## Directory Structure

```
/content/loo_temp_<dataset_name>/
├── train/
│   ├── background/
│   └── bats/
├── val/
│   ├── background/
│   └── bats/
└── test/
    ├── background/
    └── bats/
```

Each folder should contain image files named with identifiable gîte prefixes (e.g. `Jenneret_Camera_1_PHOTO_img123.jpg`).

---

## Key Modules

### Configuration
- Sets paths, batch size, learning rate, device, and seeds for reproducibility.

### Transforms and Augmentation
- Applies standard resizing and normalization.
- Augmentation includes rotation, jitter, perspective, and Gaussian blur.
- MixUp and CutMix are applied in training mode.

### Data Handling
- Uses `ImageFolder` for dataset loading.
- Implements weighted sampling to balance classes in training.

### Model Architecture
- Uses pretrained `efficientnet_b0` from `torchvision`.
- Freezes early layers (2/3) and customizes the classification head.

### Loss Function
- Implements a configurable Focal Loss to reduce the impact of easy negatives.

### Training Loop
- Supports early stopping based on F1 score improvements on validation data.
- Trains with `GradScaler` for AMP and schedules learning rate with cosine annealing.

### Threshold Optimization
- Automatically selects the decision threshold where precision ≈ recall.
- Saves optimal threshold and visualizes the ROC curve.

### Inference
- Loads the best-performing model.
- Applies threshold to test/inference images and prints classification results.

### Final Evaluation
- Computes and prints macro-averaged precision, recall, F1, and AUC on test set.
- Plots ROC curve for test data.

---

## Hardware Requirements
- Designed for use on Google Colab with an NVIDIA T4 GPU.
- Can fallback to CPU if GPU is unavailable.

---

## Outputs
- `best_model.pth`: Saved weights of the best-performing model.
- `val_blob.pt`: Cached validation logits and labels.
- `best_thr.txt`: Optimal threshold for bat classification.
- ROC plots displayed inline via `matplotlib`.

---

## Notes
- Ensure your dataset is pre-structured before training.
- Adjust `NUM_EPOCHS`, `BATCH_SIZE`, or learning rate as needed for larger datasets.
- This script is modular and can be adapted to other binary classification tasks with minimal changes.


In [ ]:
# ===========================  bat_classifier.py (EfficientNet version)  ===========================
"""
EfficientNet-B0 bat / background classifier
• Mixed-precision via torch.amp.autocast  (no deprecation warnings)
• torch.compile(), fast DataLoader, in-file Focal-Loss
• Tuned for Google-Colab’s NVIDIA T4
"""

import os
import random
import json
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder

from PIL import Image
from collections import Counter
from tqdm.auto import tqdm

from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    precision_recall_curve, roc_curve, auc
)

import matplotlib.pyplot as plt

from timm.data.mixup import Mixup

import albumentations as A
from albumentations.pytorch import ToTensorV2

# ── Reproducibility ────────────────────────────────────────────────────────────────
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = True
torch.set_float32_matmul_precision("high")

# ── Config ─────────────────────────────────────────────────────────────────────────
DATA_DIR      = "/content/loo_temp_Anthisnes_Chateau_de_Xhos_Camera_1_HIT"
TRAIN_DIR     = os.path.join(DATA_DIR, "train")
VAL_DIR       = os.path.join(DATA_DIR, "val")
TEST_DIR      = os.path.join(DATA_DIR, "test")
INFER_DIR     = TEST_DIR

OUTPUT_DIR    = "/content/efficientnet_finetuned_bats_Anthisnes_Chateau_de_Xhos_Camera_1_HIT"
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE    = 32
NUM_EPOCHS    = 1
LEARNING_RATE = 3e-5
IMAGE_SIZE    = 224

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ── AMP Setup ──────────────────────────────────────────────────────────────────────
try:
    from torch.amp import autocast, GradScaler
    AMP_KW = dict(device_type="cuda")
except ImportError:
    from torch.cuda.amp import autocast, GradScaler
    AMP_KW = {}
scaler = GradScaler()

# ── MixUp ───────────────────────────────────────────────────────────────────────────
mixup_fn = Mixup(
    num_classes=2,
    mixup_alpha=0.2,
    cutmix_alpha=1.0,
    prob=1.0,
    switch_prob=0.5,
    mode="batch",
    label_smoothing=0.0
)

# ── Albumentations pipelines ───────────────────────────────────────────────────────
train_albu = A.Compose([
    A.RandomResizedCrop(
        size=(IMAGE_SIZE, IMAGE_SIZE),   # ← use `size=` now
        scale=(0.5, 1.0),
        ratio=(0.9, 1.1),
        p=1.0
    ),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.3),
    A.RandomBrightnessContrast(
        brightness_limit=0.2,
        contrast_limit=0.2,
        p=0.3
    ),
    A.RandomGamma(gamma_limit=(80, 120), p=0.3),
    A.GaussNoise(std_range=(0.04, 0.20), p=0.2),
    A.OneOf([
        A.MotionBlur(blur_limit=5, p=1.0),
        A.MedianBlur(blur_limit=5, p=1.0),
        A.Blur(blur_limit=5, p=1.0),
    ], p=0.2),
    A.CLAHE(p=0.1),
    A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ToTensorV2(),
])

val_albu = A.Compose([
    A.Resize(                       # ← this is fine, but you can also use `size=`:
        height=IMAGE_SIZE, width=IMAGE_SIZE
    ),
    A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ToTensorV2(),
])

# ── Dataset wrapper ────────────────────────────────────────────────────────────────
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, folder, albu_transform):
        self.ds   = ImageFolder(folder, transform=None)
        self.albu = albu_transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        img, lbl = self.ds[idx]
        augmented = self.albu(image=np.array(img))
        return augmented["image"], lbl

# ── DataLoaders ───────────────────────────────────────────────────────────────────
train_set = AlbumentationsDataset(TRAIN_DIR, train_albu)
val_set   = AlbumentationsDataset(VAL_DIR,   val_albu)
test_set  = AlbumentationsDataset(TEST_DIR,  val_albu)

# compute class‐balanced sampler for train
targets      = train_set.ds.targets
class_cnt    = Counter(targets)
cnt          = torch.tensor([class_cnt[i] for i in range(len(class_cnt))], dtype=torch.float)
class_weights = (1. / cnt)
class_weights /= class_weights.sum()
sample_weights = class_weights[targets]

sampler = WeightedRandomSampler(
    sample_weights, num_samples=len(train_set)*2, replacement=True
)
num_workers = os.cpu_count() or 2

train_loader = DataLoader(
    train_set, batch_size=BATCH_SIZE, sampler=sampler,
    num_workers=num_workers, pin_memory=True,
    persistent_workers=True, prefetch_factor=4
)
val_loader = DataLoader(
    val_set,   batch_size=BATCH_SIZE*2, shuffle=False,
    num_workers=num_workers, pin_memory=True,
    persistent_workers=True
)
test_loader = DataLoader(
    test_set,  batch_size=BATCH_SIZE*2, shuffle=False,
    num_workers=num_workers, pin_memory=True,
    persistent_workers=True
)

# ── Focal Loss ────────────────────────────────────────────────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super().__init__()
        if isinstance(alpha, (list, tuple)):
            alpha = torch.tensor(alpha, dtype=torch.float32)
        self.alpha, self.gamma, self.reduction = alpha, gamma, reduction

    def forward(self, logits: torch.Tensor, targets: torch.Tensor):
        ce = F.cross_entropy(logits, targets, reduction="none")
        pt = torch.exp(-ce)

        if isinstance(self.alpha, torch.Tensor):
            if targets.dtype in (torch.int64, torch.int32):  # hard labels
                a = self.alpha.to(logits.device)[targets]
            else:  # soft labels
                a = self.alpha.mean().to(logits.device)
        else:
            a = self.alpha

        loss = a * (1 - pt) ** self.gamma * ce
        return loss.mean() if self.reduction == "mean" else loss.sum()

# ── Model, optimizer, scheduler ─────────────────────────────────────────────────
weights   = EfficientNet_B0_Weights.IMAGENET1K_V1
base_model= efficientnet_b0(weights=weights)

# freeze early blocks
total_blocks = len(base_model.features)
freeze_upto  = int(total_blocks * 2 / 3)
for i, block in enumerate(base_model.features):
    if i < freeze_upto:
        for p in block.parameters():
            p.requires_grad = False

base_model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(base_model.classifier[1].in_features, 2)
)

try:
    model = torch.compile(base_model.to(DEVICE))
except Exception:
    model = base_model.to(DEVICE)

alpha    = torch.tensor([0.05, 0.95])
loss_fn  = FocalLoss(alpha=alpha, gamma=2.0).to(DEVICE)
optimizer= AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler= CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS * len(train_loader))

# ── Training & Validation Loop ──────────────────────────────────────────────────
patience, best_f1, epochs_no_imp = 3, 0.0, 0
for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    train_loss = 0.0
    for xb, yb in tqdm(train_loader, desc=f"Train {epoch}/{NUM_EPOCHS}", unit="batch"):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        xb, yb = mixup_fn(xb, yb)
        with autocast(**AMP_KW):
            logits = model(xb)
            loss   = loss_fn(logits, yb)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        train_loss += loss.item()

    avg_train = train_loss / len(train_loader)

    model.eval()
    val_logits, val_labels, val_loss = [], [], 0.0
    with torch.no_grad():
        for xb, yb in tqdm(val_loader, desc="Valid", unit="batch", leave=False):
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            with autocast(**AMP_KW):
                out  = model(xb)
                loss = loss_fn(out, yb)
            val_loss += loss.item()
            val_logits.append(out.cpu())
            val_labels.append(yb.cpu())

    val_logits = torch.cat(val_logits)
    val_labels = torch.cat(val_labels)
    avg_val = val_loss / len(val_loader)
    preds   = val_logits.argmax(dim=1)

    f1_macro = f1_score(val_labels, preds, average="macro")
    prec_m   = precision_score(val_labels, preds, average="macro")
    rec_m    = recall_score(val_labels, preds, average="macro")

    print(
        f"Epoch {epoch:02d} │ "
        f"TrainL {avg_train:.4f} │ ValL {avg_val:.4f} │ "
        f"F1_macro {f1_macro:.4f} │ Precision {prec_m:.4f} │ Recall {rec_m:.4f}"
    )

    if f1_macro > best_f1:
        best_f1, epochs_no_imp = f1_macro, 0
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_model.pth"))
        torch.save({"logits": val_logits, "labels": val_labels},
                   os.path.join(OUTPUT_DIR, "val_blob.pt"))
    else:
        epochs_no_imp += 1
        if epochs_no_imp >= patience:
            print("Early stopping.")
            break

# ── Threshold & ROC on Validation ─────────────────────────────────────────────────
blob   = torch.load(os.path.join(OUTPUT_DIR, "val_blob.pt"))
labels = blob["labels"].numpy()
probs  = blob["logits"].softmax(1)[:,1].numpy()

prec, rec, thr   = precision_recall_curve(labels, probs)
best_thr         = float(thr[np.argmin(np.abs(prec-rec))])
with open(os.path.join(OUTPUT_DIR, "best_thr.txt"), "w") as f:
    f.write(str(best_thr))
print(f"Optimal threshold: {best_thr:.4f}")

fpr, tpr, roc_t  = roc_curve(labels, probs)
roc_auc          = auc(fpr, tpr)
idx_pt           = np.argmin(np.abs(roc_t - best_thr))
fpr_pt, tpr_pt   = fpr[idx_pt], tpr[idx_pt]

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.3f}")
plt.scatter([fpr_pt], [tpr_pt], c="red", s=60,
            label=f"Thr={best_thr:.3f}\nFPR={fpr_pt:.3f},TPR={tpr_pt:.3f}")
plt.plot([0,1],[0,1],"k--",alpha=0.4)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC - Validation")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

# ── Inference on Single Crops ─────────────────────────────────────────────────────
print("\nInference on crops …")
model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "best_model.pth")))
model.eval()
best_thr = float(open(os.path.join(OUTPUT_DIR, "best_thr.txt")).read())

for fn in os.listdir(INFER_DIR):
    if not fn.lower().endswith((".png",".jpg",".jpeg",".bmp",".tiff")):
        continue
    img = np.array(Image.open(os.path.join(INFER_DIR, fn)).convert("RGB"))
    inp = val_albu(image=img)["image"].unsqueeze(0).to(DEVICE)
    with torch.no_grad(), autocast(**AMP_KW):
        logit = model(inp)
    prob = logit.softmax(1)[0,1].item()
    pred = "bats" if prob >= best_thr else "background"
    print(f"{fn}: {pred} (Pbat={prob:.3f})")

# ── Final Evaluation on TEST_SET ──────────────────────────────────────────────────
print("\nEvaluating on TEST_DIR …")
model.eval()
test_logits, test_labels = [], []
with torch.no_grad():
    for xb, yb in tqdm(test_loader, desc="Test", unit="batch", leave=False):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        with autocast(**AMP_KW):
            out = model(xb)
        test_logits.append(out.cpu())
        test_labels.append(yb.cpu())

test_logits = torch.cat(test_logits)
test_labels = torch.cat(test_labels)
test_probs  = test_logits.softmax(1)[:,1]
test_preds  = (test_probs >= best_thr).int()

precision = precision_score(test_labels, test_preds, average="macro")
recall    = recall_score(test_labels, test_preds, average="macro")
f1_score_  = f1_score(test_labels, test_preds, average="macro")
fpr, tpr, _= roc_curve(test_labels, test_probs)
roc_auc    = auc(fpr, tpr)

print(
    f"\n TEST SET PERFORMANCE:\n"
    f"Precision: {precision:.4f} │ Recall: {recall:.4f} │ "
    f"F1_macro: {f1_score_:.4f} │ AUC: {roc_auc:.4f}"
)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f"ROC AUC = {roc_auc:.3f}")
plt.plot([0,1],[0,1],'k--',alpha=0.3)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - TEST")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()


# Final Train for end-point model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Final_train.zip /content
!unzip /content/Final_train.zip -d /content > /dev/null

In [ ]:
# ===========================  bat_classifier.py (EfficientNet version)  ===========================
"""
EfficientNet-B0 bat / background classifier
• Mixed-precision via torch.amp.autocast
• torch.compile(), fast DataLoader, in-file Focal-Loss
• Trained on the entire Final_train dataset
"""

import os
import random
import json
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder

from PIL import Image
from collections import Counter
from tqdm.auto import tqdm

from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    precision_recall_curve, roc_curve, auc
)

import matplotlib.pyplot as plt

from timm.data.mixup import Mixup

import albumentations as A
from albumentations.pytorch import ToTensorV2

# ── Reproducibility ────────────────────────────────────────────────────────────────
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = True
torch.set_float32_matmul_precision("high")

# ── Config ─────────────────────────────────────────────────────────────────────────
DATA_DIR   = "/content/Final_train"
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE    = 32
NUM_EPOCHS    = 10
LEARNING_RATE = 3e-5
IMAGE_SIZE    = 224

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ── AMP Setup ──────────────────────────────────────────────────────────────────────
try:
    from torch.amp import autocast, GradScaler
    AMP_KW = dict(device_type="cuda")
except ImportError:
    from torch.cuda.amp import autocast, GradScaler
    AMP_KW = {}
scaler = GradScaler()

# ── MixUp ───────────────────────────────────────────────────────────────────────────
mixup_fn = Mixup(
    num_classes=2,
    mixup_alpha=0.2,
    cutmix_alpha=1.0,
    prob=1.0,
    switch_prob=0.5,
    mode="batch",
    label_smoothing=0.0
)

# ── Albumentations pipelines ───────────────────────────────────────────────────────
train_albu = A.Compose([
    A.RandomResizedCrop(size=(IMAGE_SIZE, IMAGE_SIZE), scale=(0.5, 1.0), ratio=(0.9, 1.1), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.RandomGamma(gamma_limit=(80, 120), p=0.3),
    A.GaussNoise(std_range=(0.04, 0.20), p=0.2),
    A.OneOf([
        A.MotionBlur(blur_limit=5, p=1.0),
        A.MedianBlur(blur_limit=5, p=1.0),
        A.Blur(blur_limit=5, p=1.0),
    ], p=0.2),
    A.CLAHE(p=0.1),
    A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    ToTensorV2(),
])

# ── Dataset wrapper ────────────────────────────────────────────────────────────────
class AlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, folder, albu_transform):
        self.ds   = ImageFolder(folder, transform=None)
        self.albu = albu_transform

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        img, lbl = self.ds[idx]
        augmented = self.albu(image=np.array(img))
        return augmented["image"], lbl

# ── DataLoader ────────────────────────────────────────────────────────────────────
train_set = AlbumentationsDataset(TRAIN_DIR, train_albu)

# compute class‐balanced sampler
targets       = train_set.ds.targets
class_cnt     = Counter(targets)
cnt           = torch.tensor([class_cnt[i] for i in range(len(class_cnt))], dtype=torch.float)
class_weights = (1. / cnt)
class_weights /= class_weights.sum()
sample_weights = class_weights[targets]

sampler = WeightedRandomSampler(
    sample_weights, num_samples=len(train_set), replacement=True
)

num_workers = os.cpu_count() or 2
train_loader = DataLoader(
    train_set, batch_size=BATCH_SIZE, sampler=sampler,
    num_workers=num_workers, pin_memory=True,
    persistent_workers=True, prefetch_factor=4,
    drop_last=True,
)
# ── Focal Loss ────────────────────────────────────────────────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction="mean"):
        super().__init__()
        if isinstance(alpha, (list, tuple)):
            alpha = torch.tensor(alpha, dtype=torch.float32)
        self.alpha, self.gamma, self.reduction = alpha, gamma, reduction

    def forward(self, logits: torch.Tensor, targets: torch.Tensor):
        ce = F.cross_entropy(logits, targets, reduction="none")
        pt = torch.exp(-ce)

        if isinstance(self.alpha, torch.Tensor):
            if targets.dtype in (torch.int64, torch.int32):
                a = self.alpha.to(logits.device)[targets]
            else:  # soft labels
                a = self.alpha.mean().to(logits.device)
        else:
            a = self.alpha

        loss = a * (1 - pt) ** self.gamma * ce
        return loss.mean() if self.reduction == "mean" else loss.sum()
# ── Model, optimizer, scheduler ─────────────────────────────────────────────────
weights    = EfficientNet_B0_Weights.IMAGENET1K_V1
base_model = efficientnet_b0(weights=weights)

# freeze early blocks
total_blocks = len(base_model.features)
freeze_upto  = int(total_blocks * 2 / 3)
for i, block in enumerate(base_model.features):
    if i < freeze_upto:
        for p in block.parameters():
            p.requires_grad = False

base_model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(base_model.classifier[1].in_features, 2)
)

try:
    model = torch.compile(base_model.to(DEVICE))
except Exception:
    model = base_model.to(DEVICE)

alpha     = torch.tensor([0.05, 0.95])
loss_fn   = FocalLoss(alpha=alpha, gamma=2.0).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS * len(train_loader))

# ── Training Loop ────────────────────────────────────────────────────────────────
for epoch in range(1, NUM_EPOCHS+1):
    model.train()
    train_loss = 0.0
    for xb, yb in tqdm(train_loader, desc=f"Train {epoch}/{NUM_EPOCHS}", unit="batch"):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        xb, yb = mixup_fn(xb, yb)
        with autocast(**AMP_KW):
            logits = model(xb)
            loss   = loss_fn(logits, yb)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        train_loss += loss.item()

    avg_train = train_loss / len(train_loader)
    print(f"Epoch {epoch:02d} │ Train Loss {avg_train:.4f}")

    # save checkpoint each epoch
    torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, f"model_epoch{epoch}.pth"))

# ── Save final model ──────────────────────────────────────────────────────────────
torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "best_model.pth"))
print("Training complete. Model saved to", OUTPUT_DIR)


In [ ]:
# prompt: save the best_model.pth in the drive
import shutil
import os
# Copy the trained model to Google Drive
drive_output_path = "/content/drive/MyDrive/best_model.pth"
shutil.copyfile(os.path.join(OUTPUT_DIR, "best_model.pth"), drive_output_path)
print(f"Saved best_model.pth to {drive_output_path}")
